In [1]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [ ]:
!echo "Downloading Images"
!curl -L -o kagglecatsanddogs_5340.zip --progress-bar https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip kagglecatsanddogs_5340.zip
!rm kagglecatsanddogs_5340.zip
!ls

In [3]:
baseDir = "/content/PetImages"
catDir = os.path.join(baseDir, 'Cat')
dogDir = os.path.join(baseDir, 'Dog')
print('total training cat images:', len(os.listdir(catDir)))
print('total training dog images:', len(os.listdir(dogDir)))

total training cat images: 12501
total training dog images: 12501


In [11]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [26]:
data = []

cats = [os.path.join(dp, f) for dp, dn, filenames in os.walk(catDir) for f in filenames]
print(cats[0:10])
for catPath in cats[:500]:
  img, x = get_image(catPath)
  data.append({'x':np.array(x[0]), 'y': 0})

dogs = [os.path.join(dp, f) for dp, dn, filenames in os.walk(dogDir) for f in filenames]
print(dogs[:10])
for dogPath in dogs[:500]:
  img, x = get_image(dogPath)
  data.append({'x':np.array(x[0]), 'y': 1})

import random
random.shuffle(data)


['/content/PetImages/Cat/10697.jpg', '/content/PetImages/Cat/297.jpg', '/content/PetImages/Cat/7269.jpg', '/content/PetImages/Cat/12199.jpg', '/content/PetImages/Cat/2286.jpg', '/content/PetImages/Cat/10048.jpg', '/content/PetImages/Cat/5994.jpg', '/content/PetImages/Cat/2740.jpg', '/content/PetImages/Cat/6894.jpg', '/content/PetImages/Cat/6625.jpg']
['/content/PetImages/Dog/10697.jpg', '/content/PetImages/Dog/297.jpg', '/content/PetImages/Dog/7269.jpg', '/content/PetImages/Dog/12199.jpg', '/content/PetImages/Dog/2286.jpg', '/content/PetImages/Dog/10048.jpg', '/content/PetImages/Dog/5994.jpg', '/content/PetImages/Dog/2740.jpg', '/content/PetImages/Dog/6894.jpg', '/content/PetImages/Dog/6625.jpg']


In [4]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

553467096/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [5]:
inp = vgg.input
new_classification_layer = Dense(2, activation='softmax') # 2 classes (Caes e gatos)
out = new_classification_layer(vgg.layers[-2].output)
model_new = Model(inp, out)

for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [27]:
train, val, test = data[:700], data[701:850], data[851:1000]
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [28]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, 2)
y_val = keras.utils.to_categorical(y_val, 2)
y_test = keras.utils.to_categorical(y_test, 2)
print(y_test.shape)

(149, 2)


In [29]:
print("finished loading %d images from %d categories"%(len(data), 2))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)

finished loading 1000 images from 2 categories
train / validation / test split: 700, 149, 149
training data shape:  (700, 224, 224, 3)
training labels shape:  (700, 2)


In [ ]:
history = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))

Epoch 1/10
6/6 [==============================] - 568s 98s/step - loss: 1.0203 - accuracy: 0.4771 - val_loss: 0.7324 - val_accuracy: 0.4765
Epoch 2/10
6/6 [==============================] - 550s 95s/step - loss: 0.7445 - accuracy: 0.5243 - val_loss: 0.7633 - val_accuracy: 0.4832
Epoch 3/10
3/6 [==============>...............] - ETA: 3:48 - loss: 0.7187 - accuracy: 0.5417